<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with TensorFlow Lite Model Maker

In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Android figurines and how to put the model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.7 MB/s eta 0:

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### Prepare the dataset

We start with downloading the dataset.

In [ ]:
# Uncomment and run only ONCE. Alternatively, change link to other Github repo or weblink.
!wget https://github.com/Talonee/Portal/raw/main/pi/grass.zip
!unzip -q grass.zip


--2023-02-25 01:22:13--  https://github.com/Talonee/Portal/raw/main/pi/grass.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Talonee/Portal/main/pi/grass.zip [following]
--2023-02-25 01:22:13--  https://raw.githubusercontent.com/Talonee/Portal/main/pi/grass.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59741808 (57M) [application/zip]
Saving to: ‘grass.zip’

grass.zip           100%[===================>]  56.97M   290MB/s    in 0.2s    

2023-02-25 01:22:13 (290 MB/s) - ‘grass.zip’ saved [59741808/59741808]



## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
# Import modules to convert images
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
import re

dir = "train/"
print(listdir(dir))

for f in listdir(dir):
    if (isfile(f"{dir}{f}") and re.search("jpg$", f)):
        # importing the image 
        im = Image.open(f"{dir}{f}")
        if not im.mode == 'RGB':
            print(f"Ayo what the fuck{dir}{f}")
            # re-convert & save to jpg, wget downloads from google isn't recognized as JPEGs
            im.convert('RGB').save(f"{dir}{f}","JPEG")       

['13.jpg', '18.jpg', '93.jpg', '34.jpg', '78.xml', '32.jpg', '49.xml', '74.jpg', '76.xml', '50.jpg', '59.xml', '96.jpg', '9.xml', '82.xml', '99.xml', '22.jpg', '66.xml', '80.jpg', '99.jpg', '62.xml', '67.xml', '64.xml', '101.jpg', '73.jpg', '26.jpg', '35.xml', '79.jpg', '102.xml', '84.jpg', '101.xml', '14.xml', '94.xml', '80.xml', '31.xml', '78.jpg', '70.xml', '30.jpg', '94.jpg', '10.jpg', '26.xml', '70.jpg', '49.jpg', '7.jpg', '85.jpg', '7.xml', '100.jpg', '28.jpg', '73.xml', '69.xml', '29.xml', '56.jpg', '16.jpg', '9.jpg', '76.jpg', '34.xml', '21.jpg', '75.jpg', '86.xml', '67.jpg', '12.xml', '79.xml', '14.jpg', '69.jpg', '90.jpg', '63.xml', '32.xml', '64.jpg', '82.jpg', '83.xml', '11.xml', '62.jpg', '13.xml', '12.jpg', '47.xml', '16.xml', '50.xml', '29.jpg', '28.xml', '56.xml', '31.jpg', '5.xml', '3.xml', '66.jpg', '4.jpg', '68.xml', '19.jpg', '63.jpg', '35.jpg', '83.jpg', '58.jpg', '21.xml', '4.xml', '19.xml', '68.jpg', '8.jpg', '60.xml', '5.jpg', '100.xml', '86.jpg', '102.jpg', '93

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'train',
    'train',
    ['grass']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'validate',
    'validate',
    ['grass']
)

# See how many images are available in each set: train, validate.
print(f"Train: {len(train_data)} images\nValidate: {len(val_data)} images")

Train: 61 images
Validate: 18 images


### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
model_id = 0
spec = model_spec.get(f'efficientdet_lite{model_id}')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
15/15 [==============================] - 59s 972ms/step - det_loss: 1.7360 - cls_loss: 1.1091 - box_loss: 0.0125 - reg_l2_loss: 0.0630 - loss: 1.7990 - learning_rate: 0.0065 - gradient_norm: 2.3434 - val_det_loss: 1.5882 - val_cls_loss: 1.0756 - val_box_loss: 0.0103 - val_reg_l2_loss: 0.0630 - val_loss: 1.6512
Epoch 2/20
15/15 [==============================] - 12s 826ms/step - det_loss: 1.5982 - cls_loss: 1.0174 - box_loss: 0.0116 - reg_l2_loss: 0.0630 - loss: 1.6612 - learning_rate: 0.0049 - gradient_norm: 2.8597 - val_det_loss: 1.4225 - val_cls_loss: 0.9681 - val_box_loss: 0.0091 - val_reg_l2_loss: 0.0630 - val_loss: 1.4855
Epoch 3/20
15/15 [==============================] - 11s 766ms/step - det_loss: 1.4012 - cls_loss: 0.8946 - box_loss: 0.0101 - reg_l2_loss: 0.0630 - loss: 1.4642 - learning_rate: 0.0048 - gradient_norm: 3.7608 - val_det_loss: 1.1385 - val_cls_loss: 0.7457 - val_box_loss: 0.0079 - val_reg_l2_loss: 0.0630 - val_loss: 1.2016
Epoch 4/20
15/15 [=============

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.5904379,
 'AP50': 0.84582746,
 'AP75': 0.7335831,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.59048575,
 'ARmax1': 0.6,
 'ARmax10': 0.65555555,
 'ARmax100': 0.6722222,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.6722222,
 'AP_/grass': 0.5904379}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [ ]:
tflite_name=f'grass{model_id}.tflite'
model.export(export_dir='.', tflite_filename=tflite_name)

### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
# Highest performance: 67%
model.evaluate_tflite(tflite_name, val_data)

18/18 [==============================] - 57s 3s/step



{'AP': 0.5651574,
 'AP50': 0.8394497,
 'AP75': 0.7116337,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.5651574,
 'ARmax1': 0.59444445,
 'ARmax10': 0.6166667,
 'ARmax100': 0.6166667,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.6166667,
 'AP_/grass': 0.5651574}

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download(tflite_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>